""In this extended model, we will see if we can correct the overfitting, and try a couple of different approaches to get the best model.""

In [1]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            %matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import scale

print("Loaded")

Loaded


In [2]:
df = pd.read_csv (r"C:\Users\Laura-Black\Documents\PhD\Data Scientist\Springboard\Capstone Projects\Drug Abuse ED Visits\Output files\Capstone1DataWrangling.csv", 
                     index_col=0, low_memory=False)

In [3]:
print('The number of data points =\n', df.count())
print('The number of features = ',len(df.columns))
print('The target is Case Type.')
print('The number of distinct values in the target = ', df['CASETYPE'].nunique())
print('They are: ',df.CASETYPE.unique())
print('The relative size of each class = \n', df['CASETYPE'].value_counts(normalize=True))

The number of data points =
 METRO           229211
STRATA          229211
PSU             229211
REPLICATE       229211
CASEWGT         229211
                 ...  
ALCOHOL         229211
NONALCILL       229211
PHARMA          229211
NONMEDPHARMA    229211
ALLABUSE        229211
Length: 86, dtype: int64
The number of features =  86
The target is Case Type.
The number of distinct values in the target =  8
They are:  ['Other' 'Adverse Reaction' 'Seeking Detox' 'Suicide Attempt'
 'Overmedication' 'Accidential Ingestion' 'Alcohol (Age<21)'
 'Malicious Poisoning']
The relative size of each class = 
 Adverse Reaction         0.384345
Other                    0.382303
Overmedication           0.079167
Seeking Detox            0.064748
Suicide Attempt          0.039409
Alcohol (Age<21)         0.032376
Accidential Ingestion    0.014192
Malicious Poisoning      0.003460
Name: CASETYPE, dtype: float64


In [4]:
print(df.head())
print(df.info())

              METRO  STRATA  PSU  REPLICATE   CASEWGT  PSUFRAME   AGECAT  \
CASEID                                                                     
1          New York      25  108          2  0.942635         3    18-20   
2          New York      29  129          2  5.992011         9     > 65   
3       Date County       7   25          1  4.723172         6     > 65   
4           Phoenix       8   29          2  4.080147         6  06-11\t   
5            Boston      22   94          2  5.177709        10    25-29   

           SEX      RACE  YEAR  ...       DRUGID_22        ROUTE_22  \
CASEID                          ...                                   
1         Male     Black  2011  ...  Not Applicable  Not Applicable   
2         Male  Hispanic  2011  ...  Not Applicable  Not Applicable   
3       Female     Black  2011  ...  Not Applicable  Not Applicable   
4         Male  Hispanic  2011  ...  Not Applicable  Not Applicable   
5         Male  Hispanic  2011  ...  Not 

""Now, we will create the design matrix (X) and the target vector (y) for
the associated classification problem.""

In [5]:
x_split = df.drop('CASETYPE', axis=1)

X = pd.get_dummies(x_split,drop_first=True)

In [6]:
y = df['CASETYPE'].values

""Next, we will split the data into test and training sets.""

In [7]:
X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(X, y, test_size = 0.5, random_state=42, stratify=y)

In [8]:
print(X_train_all.info())
print(y_train_all.shape)
print(y_train_all.dtype)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114605 entries, 16884 to 36951
Columns: 7621 entries, STRATA to ALLABUSE_Not All Misuse & Abuse Episode
dtypes: float64(1), int64(7), uint8(7613)
memory usage: 839.9 MB
None
(114605,)
object


In [9]:
print('The number of data points =\n', X_train_all.count())
print('The number of features = ',len(X_train_all.columns))

The number of data points =
 STRATA                                     114605
PSU                                        114605
REPLICATE                                  114605
CASEWGT                                    114605
PSUFRAME                                   114605
                                            ...  
ALCOHOL_Yes                                114605
NONALCILL_No Illicit Drugs                 114605
PHARMA_Pharmaceuticals                     114605
NONMEDPHARMA_Non_Medical Pharma            114605
ALLABUSE_Not All Misuse & Abuse Episode    114605
Length: 7621, dtype: int64
The number of features =  7621


In [10]:
from collections import Counter

c_train = Counter(y_train_all)
print([(i, c_train[i] / len(y_train_all)) for i in c_train])
print('Length of y = ',len(y_train_all))

[('Adverse Reaction', 0.38434623271235985), ('Other', 0.38230443697918937), ('Overmedication', 0.07916757558570743), ('Seeking Detox', 0.06474412111164435), ('Alcohol (Age<21)', 0.032372060555822174), ('Suicide Attempt', 0.03940491252563152), ('Accidential Ingestion', 0.01419658828148859), ('Malicious Poisoning', 0.003464072248156712)]
Length of y =  114605


""The first attempt to run SMOTE on the dataset produced a memory error on the computer. Therefore, it was decided to reduce the data set using train_test_split.""

In [11]:
X_train_all.reset_index(drop = True, inplace = True)
y_train_all_df = pd.DataFrame(y_train_all)
y_train_all_df.reset_index(drop = True, inplace = True)

In [12]:
X_train_reduce, X_test_reduce, y_train_reduce, y_test_reduce = train_test_split(X_train_all, y_train_all, test_size = 0.5, random_state=0, stratify=y_train_all)

In [13]:
from collections import Counter

c_train = Counter(y_train_reduce)
[(i, c_train[i] / len(y_train_reduce)) for i in c_train]


[('Other', 0.3823077728526055),
 ('Adverse Reaction', 0.3843495864018708),
 ('Seeking Detox', 0.06474468604935256),
 ('Overmedication', 0.0791595406792084),
 ('Accidential Ingestion', 0.014187986457715263),
 ('Malicious Poisoning', 0.0034728281735367005),
 ('Suicide Attempt', 0.03940525636103452),
 ('Alcohol (Age<21)', 0.03237234302467628)]

In [14]:
c_test = Counter(y_test_reduce)
[(i, c_test[i] / len(y_test_reduce)) for i in c_test]

[('Adverse Reaction', 0.38434287908137443),
 ('Other', 0.3823011011639879),
 ('Overmedication', 0.07917561035198856),
 ('Suicide Attempt', 0.03940456869622882),
 ('Alcohol (Age<21)', 0.03237177809189746),
 ('Seeking Detox', 0.06474355618379492),
 ('Accidential Ingestion', 0.01420518995515069),
 ('Malicious Poisoning', 0.0034553164755771948)]

""Now we will run the reduced data set through SMOTE to see if we can get better results in our model.""

In [15]:
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE().fit_resample(X_train_reduce, y_train_reduce)
print(sorted(Counter(y_resampled).items()))

[('Accidential Ingestion', 22024), ('Adverse Reaction', 22024), ('Alcohol (Age<21)', 22024), ('Malicious Poisoning', 22024), ('Other', 22024), ('Overmedication', 22024), ('Seeking Detox', 22024), ('Suicide Attempt', 22024)]


In [16]:
X_train = X_resampled
y_train = y_resampled
X_test = X_test_reduce
y_test = y_test_reduce

In [17]:
# Create the classifier: logreg
logreg = LogisticRegression(multi_class='multinomial',solver='lbfgs', max_iter=1000000000)

# Fit the classifier to the training data
logreg.fit(X_train, y_train)

# Predict the labels of the training and test sets
y_pred_test = logreg.predict(X_test)
y_pred_train = logreg.predict(X_train)

# Compute and print the confusion matrices and classification reports for the Training set
print('Train Set')
print(confusion_matrix(y_train, y_pred_train))
print(classification_report(y_train, y_pred_train))
print(logreg.score(X_train,y_train))

# Compute and print the confusion matrices and classification reports for the Test set
print('/n Test Set')
print(confusion_matrix(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))
print(logreg.score(X_test,y_test))


C:\Users\Laura-Black\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Train Set
[[18635  2906     0    64   128     0     3   288]
 [ 1896 18601     0     3     2     0   140  1382]
 [    0     0 22024     0     0     0     0     0]
 [    0     0     0 20168  1293   540    23     0]
 [    0     0     0  1336 14353  3545  2711    79]
 [    0     0     0   370   273 21381     0     0]
 [   23   774    37    70  2104     0 17698  1318]
 [  189  1692    92    14   531     0  1265 18241]]
                       precision    recall  f1-score   support

Accidential Ingestion       0.90      0.85      0.87     22024
     Adverse Reaction       0.78      0.84      0.81     22024
     Alcohol (Age<21)       0.99      1.00      1.00     22024
  Malicious Poisoning       0.92      0.92      0.92     22024
                Other       0.77      0.65      0.71     22024
       Overmedication       0.84      0.97      0.90     22024
        Seeking Detox       0.81      0.80      0.81     22024
      Suicide Attempt       0.86      0.83      0.84     22024

            

print('Train Set')
print(confusion_matrix(y_train, y_pred_train))
print(classification_report(y_train, y_pred_train))
print(logreg.score(X_train,y_train))

""Both the training set and the test set show an overall accuracy of 0.77. The test set has the following results:
1. Accidental Ingestion: 22% of the predictions were correct. 
2. Adverse Reaction: 97% of the predictions were correct.
3. Alcohol (<21): 99% of the predictions were correct. Because there are so few of these cases, it is odd that the precision is high. There is a possibility of overfitting, but the recall is not good so there is no way to correct it.
4. Malicious Poisoning: 9% of the predictions were correct. The accuracy of this class is affected by the low number of cases in the data set. More cases in this class would have resulted in a higher prediction rate.
5. Other: 95% of the predictions were correct.
6. Overmedication: 55% of the predictions were correct.
7. Seeking Detox: 47% of the predictions were correct.
8. Suicide Attempt: 47% of the predictions were correct.

In [18]:
import pickle
filenameX = 'CP1_Model_X_train_reduce'
outfile = open(filenameX,'w+b')
pickle.dump(X_train,outfile, protocol=4)
outfile.close()

#X_train, X_test, y_train, y_test

In [19]:
filenameXtest = 'CP1_Model_X_test_reduce'
outfile = open(filenameXtest,'w+b')
pickle.dump(X_test,outfile)
outfile.close()

In [20]:
filenameYtest = 'CP1_Model_y_test_reduce'
outfile = open(filenameYtest,'wb')
pickle.dump(y_test,outfile)
outfile.close()

In [21]:
filenameYtrain = 'CP1_Model_y_train_reduce'
outfile = open(filenameYtrain,'wb')
pickle.dump(y_train,outfile)
outfile.close()

""After seeing the results of the reduced data set on the Logistic Regression model, it was decided to use Random Forest to see if the results improve.""

In [22]:
#Random Forest
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100, oob_score=True)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred_test_rf=clf.predict(X_test)

In [24]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_test_rf))

Accuracy: 0.8604436067919655


In [25]:
print('Random Forest Test Set')
print(confusion_matrix(y_test, y_pred_test_rf))
print(classification_report(y_test, y_pred_test_rf))
print(clf.score(X_test,y_test))

Random Forest Test Set
[[   32   754     0     0    25     0     0     3]
 [    5 21772     0     0    75     8    29   135]
 [    0     0  1854     0     0     0     1     0]
 [    0     0     0    53   138     7     0     0]
 [    0     2     0     8 20427  1180   290     0]
 [    0    19     0     2  1528  2985     0     3]
 [    0   430     6     0  1566     4  1645    59]
 [    2  1178    10     0   421    20    89   538]]
                       precision    recall  f1-score   support

Accidential Ingestion       0.82      0.04      0.08       814
     Adverse Reaction       0.90      0.99      0.94     22024
     Alcohol (Age<21)       0.99      1.00      1.00      1855
  Malicious Poisoning       0.84      0.27      0.41       198
                Other       0.84      0.93      0.89     21907
       Overmedication       0.71      0.66      0.68      4537
        Seeking Detox       0.80      0.44      0.57      3710
      Suicide Attempt       0.73      0.24      0.36      2258


In [26]:
print(clf.oob_score_)

0.96606542862332


""The results of the Random Forest model indicate that there is some overfitting. Therefore, it was decided to do a Randomized Search to tune the hyperparameters."" 

In [32]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

distributions = {'n_estimators': [100, 300, 500, 750, 800, 1200], 'max_depth': [5,10,15,20,30], 'max_features': [10,20,30,40,50,60,70,80,90,100], 'random_state': [42]}
rscv = RandomizedSearchCV(clf, distributions, random_state=0)
search = rscv.fit(X_train, y_train)
search.best_params_

C:\Users\Laura-Black\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


{'random_state': 42,
 'n_estimators': 1200,
 'max_features': 100,
 'max_depth': 20}

In [33]:
clf=RandomForestClassifier(n_estimators=1200, oob_score=True, random_state=42, max_features=100, max_depth=20)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred_test_rf=clf.predict(X_test)

In [34]:
print('Random Forest Tuned')
print(confusion_matrix(y_test, y_pred_test_rf))
print(classification_report(y_test, y_pred_test_rf))
print(clf.score(X_test,y_test))

Random Forest Tuned
[[  245   501     0     0    25     0     0    43]
 [  755 20332     0    35    64     8    70   760]
 [    0     0  1855     0     0     0     0     0]
 [    0     0     0    79    89    30     0     0]
 [    0     0     1   204 16476  3772  1452     2]
 [    0    12     0    90   179  4252     0     4]
 [   19   388     6    11   917     0  2228   141]
 [  103   779    10    17   306     3   151   889]]
                       precision    recall  f1-score   support

Accidential Ingestion       0.22      0.30      0.25       814
     Adverse Reaction       0.92      0.92      0.92     22024
     Alcohol (Age<21)       0.99      1.00      1.00      1855
  Malicious Poisoning       0.18      0.40      0.25       198
                Other       0.91      0.75      0.82     21907
       Overmedication       0.53      0.94      0.67      4537
        Seeking Detox       0.57      0.60      0.59      3710
      Suicide Attempt       0.48      0.39      0.43      2258

  

""Neither of the models that were corrected for overfitting: the Logistic Rgression with the Reduced Data Set and the Random Forest with Hyperparameter Tuning are as accurate as we would like them to be. The over all accuracy of the LR model is 0.77 and for the RF model is 0.81. They both predict the cases with smaller numbers poorly, and both overfit the Alcohol (Age<21). 

It would be preferrable to do some more fine tuning and re-run the models; however, time does not permit us to do it right now. ""